# SQL Data Analysis Project
## Dataset: Northwind Traders Database (Kaggle)
## Tool: SQLite in Jupyter Notebook
---

In [ ]:
import sqlite3
import pandas as pd

# Connect to Database (northwind.db downloaded from Kaggle)
conn = sqlite3.connect("northwind.db")

# Helper function to run SQL and display as DataFrame
def run_query(query):
    return pd.read_sql_query(query, conn)

In [ ]:
# 1. List customers from USA
query1 = """
SELECT CustomerID, CompanyName, ContactName, Country
FROM Customers
WHERE Country = 'USA';
"""
run_query(query1)

In [ ]:
# 2. Orders sorted by Freight (like total amount)
query2 = """
SELECT OrderID, CustomerID, OrderDate, Freight
FROM Orders
ORDER BY Freight DESC
LIMIT 10;
"""
run_query(query2)

In [ ]:
# 3. Total sales per customer
query3 = """
SELECT c.CustomerID, c.CompanyName, SUM(od.UnitPrice * od.Quantity) AS TotalSpent
FROM Customers c
JOIN Orders o ON c.CustomerID = o.CustomerID
JOIN [Order Details] od ON o.OrderID = od.OrderID
GROUP BY c.CustomerID, c.CompanyName
ORDER BY TotalSpent DESC
LIMIT 10;
"""
run_query(query3)

In [ ]:
# 4. Products purchased by each customer
query4 = """
SELECT c.CompanyName AS Customer, p.ProductName, SUM(od.Quantity) AS QuantityBought
FROM Customers c
JOIN Orders o ON c.CustomerID = o.CustomerID
JOIN [Order Details] od ON o.OrderID = od.OrderID
JOIN Products p ON od.ProductID = p.ProductID
GROUP BY c.CompanyName, p.ProductName
ORDER BY QuantityBought DESC
LIMIT 10;
"""
run_query(query4)

In [ ]:
# 5. Customers who spent above average
query5 = """
SELECT CompanyName
FROM Customers
WHERE CustomerID IN (
    SELECT o.CustomerID
    FROM Orders o
    JOIN [Order Details] od ON o.OrderID = od.OrderID
    GROUP BY o.CustomerID
    HAVING SUM(od.UnitPrice * od.Quantity) >
           (SELECT AVG(TotalSpent)
            FROM (
                SELECT SUM(od.UnitPrice * od.Quantity) AS TotalSpent
                FROM Orders o
                JOIN [Order Details] od ON o.OrderID = od.OrderID
                GROUP BY o.CustomerID
            ) AS sub
           )
);
"""
run_query(query5)

In [ ]:
# 6. Create a view for product sales
query6 = """
CREATE VIEW IF NOT EXISTS ProductSales AS
SELECT p.ProductID, p.ProductName, SUM(od.UnitPrice * od.Quantity) AS TotalSales
FROM Products p
JOIN [Order Details] od ON p.ProductID = od.ProductID
GROUP BY p.ProductID, p.ProductName;
"""
conn.execute(query6)

# Query the view
run_query("SELECT * FROM ProductSales ORDER BY TotalSales DESC LIMIT 10;")

In [ ]:
# 7. Optimization with Index
conn.execute("CREATE INDEX IF NOT EXISTS idx_order_date ON Orders(OrderDate);")

query7 = """
SELECT OrderID, CustomerID, OrderDate, Freight
FROM Orders
WHERE OrderDate > '1997-01-01'
ORDER BY OrderDate ASC
LIMIT 10;
"""
run_query(query7)

In [ ]:
# Close connection
conn.close()